In [1]:
import pickle

import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

from pathlib import Path

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
data_dir = Path.cwd() / "data"
sample_file_path = "sample_000000.pkl"
sample_file_path = data_dir / sample_file_path

with open(sample_file_path, "rb") as f:
    sample = pickle.load(f)

print("Sample keys:", sample.keys())

Sample keys: dict_keys(['frame_id', 'gt_boxes', 'points', 'flip_x', 'noise_rot', 'noise_scale', 'lidar_aug_matrix', 'use_lead_xyz', 'image_shape'])


In [3]:
points = sample['points']

print("Points shape:", points.shape)
print('Min/Max values for all dimensions')
min_x, max_x = np.min(points[:, 0]), np.max(points[:, 0])
min_y, max_y = np.min(points[:, 1]), np.max(points[:, 1])
min_z, max_z = np.min(points[:, 2]), np.max(points[:, 2])
print(f"X: ({min_x:.2f}, {max_x:.2f})")
print(f"Y: ({min_y:.2f}, {max_y:.2f})")
print(f"Z: ({min_z:.2f}, {max_z:.2f})")

Points shape: (16384, 4)
Min/Max values for all dimensions
X: (5.25, 63.28)
Y: (-15.11, 29.17)
Z: (-2.43, 2.02)


In [4]:
CLASS_COLORS = {
    1: (1, 0, 0),    # Car     (red)
    2: (0, 1, 0),    # Pedestrian (green)
    3: (0, 0, 1)     # Cyclist (blue)
}

def get_point_colors(z):
    z_norm = (z - z.min()) / (z.max() - z.min() + 1e-6)
    colors = plt.cm.jet(z_norm)[:, :3]
    return colors

def create_point_cloud(points):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    colors = get_point_colors(points[:, 2])
    pcd.colors = o3d.utility.Vector3dVector(colors)
    return pcd

def get_gt_oriented_bounding_boxes(gt_boxes):
    boxes = []
    for box in gt_boxes:
        x, y, z, dx, dy, dz, heading, class_idx = box
        class_idx = int(class_idx)
        obb = o3d.geometry.OrientedBoundingBox(
            center=[0, 0, 0],
            R=o3d.geometry.get_rotation_matrix_from_xyz([0, 0, heading]),
            extent=[dx, dy, dz]
        )
        obb.translate([x, y, z])
        obb.color = CLASS_COLORS.get(class_idx, (1, 1, 1))
        boxes.append(obb)
    return boxes


points = sample['points'][:, :3]
pcd = create_point_cloud(points)
gt_boxes = get_gt_oriented_bounding_boxes(sample['gt_boxes'])
o3d.visualization.draw_geometries([pcd, *gt_boxes])